## NYC Taxi Uber/Lyft Data Transformation

Let's this a bit cleaner shall we?

https://www.nyc.gov/site/tlc/businesses/high-volume-for-hire-services.page

## Get Dask cluster close to the data

In [ ]:
import coiled
cluster = coiled.Cluster(
    n_workers=10,
    name="nyc-taxi-uber-lyft",
    package_sync=True,
    backend_options={"region": "us-east-1"},
    worker_memory="64 GiB",
)

In [ ]:
from dask.distributed import Client
client = Client(cluster)

## Inspect data

In [ ]:
import s3fs

s3 = s3fs.S3FileSystem()
s3.glob("nyc-tlc/trip data/fhvhv_tripdata_*.parquet")

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet(
    "s3://nyc-tlc/trip data/fhvhv_tripdata_*.parquet",
)
df

In [ ]:
df.partitions[0].memory_usage(deep=True).compute()

In [ ]:
df.dtypes

## Two problems!

1.  Massive partitions!
2.  Inefficient data types!
    -   Strings
    -   Categories

## Convert dtypes!

In [ ]:
import pandas as pd
import numpy as np

yes_no = pd.CategoricalDtype(categories=["Y", "N"])

In [ ]:
conversions = {}
for column, dtype in df.dtypes.items():
    if dtype == "object":
        conversions[column] = "string[pyarrow]"
    if dtype == "float64":
        conversions[column] = "float32"
    if dtype == "int64": 
        conversions[column] = "int32"
    if "flag" in column:
        conversions[column] = yes_no
        
conversions

In [ ]:
df = df.astype(conversions)

## Repartition to smaller chunks!

In [ ]:
df = df.persist()

df = df.repartition(partition_size="128MB").persist()

In [ ]:
import dask

dask.util.format_bytes(
    df.memory_usage(deep=True).compute()
)

## Sort 

In [ ]:
df = df.set_index("request_datetime").persist()

## Pretty close to one-day partitioning

In [ ]:
df.divisions

In [ ]:
df = df.repartition(freq="1d")

In [ ]:
s3.mkdir("/oss-shared-scratch/mrocklin")

In [ ]:
divisions = df.divisions

def name_function(index: int) -> str:
    return str(divisions[index].date()) + ".parquet"

name_function(divisions[0])

## Save!

In [ ]:
df.to_parquet(
    "s3://oss-shared-scratch/mrocklin/nyc-taxi-fhv/", 
    name_function=name_function,
)

In [ ]:
s3.ls("/oss-shared-scratch/mrocklin/nyc-taxi-fhv")[:20]

## How is it to read this data?

In [ ]:
df = dd.read_parquet(
    "s3://oss-shared-scratch/mrocklin/nyc-taxi-fhv", 
    use_nullable_dtypes=True
).persist()

df.dtypes

In [ ]:
import dask

dask.utils.format_bytes(
    df.memory_usage(deep=True).sum().compute()
)

In [ ]:
df.memory_usage_per_partition(deep=True).compute().apply(dask.utils.format_bytes)